In [37]:
import os
import sys
import pandas as pd
import geopandas as gpd
from tqdm import tqdm
import pandas as pd
from tqdm import tqdm
import numpy as np
from scipy.stats import pearsonr, spearmanr
from datetime import datetime
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from importlib import reload
import matplotlib.pyplot as plt
from suncalc import get_position, get_times
from datetime import datetime, timezone

In [4]:
sys.path.append("../../src")
import main

In [5]:
def calc_diurnal(hid, radius=300, season='year'):

    temp =  pd.read_csv('/Users/lisawink/Documents/paper1/data/gap_filled_data_ta_rh.csv')
    temp['datetime_UTC']=pd.to_datetime(temp['datetime_UTC'])
    temp = temp[temp['variable']=='Ta_deg_C']
    temp = temp[temp['datetime_UTC'].dt.date.isin(hid['datetime_UTC'].dt.date)]
    temp['datetime_UTC'] = temp['datetime_UTC'].astype(str)
    temp = temp.pivot(index='station_id', columns='datetime_UTC', values='value')

    params = gpd.read_parquet(f'/Users/lisawink/Documents/paper1/data/processed_data/processed_station_params_{radius}.parquet')
    params = params.set_index('station_id')
    to_remove = ['station_no','station_name','station_long_name','station_type','station_lat','station_lon','station_elevation','mounting_structure','sky_view_factor','dominant_land_use','local_climate_zone','urban_atlas_class','urban_atlas_code','geometry','SVF_3D']
    params = params.drop(to_remove, axis=1)
    vars = params.columns

    vars = ['BuAre_sum', 'BuVol_3D_sum', 'BuEWA_3D_sum', 'BuIBD', 'BuAdj', 'BuSWR_3D_median', 'BuHt_wmean','StrHW_median','SVF_3D_mean', 'BuERI_wmean', 'StrClo400_median']

    for var in vars:
        params_v = params[[var]]

        stats_dict = {}

        for time in temp.columns:
            params_t = params_v.merge(temp[time], left_on='station_id', right_on='station_id',how='inner')
            stats = main.calculate_statistics(params_t, time)
            stats.index = stats['Parameter']
            stats['Time'] = time
            stats_dict[time] = stats
        pd.concat(stats_dict).to_csv(f'/Users/lisawink/Documents/paper1/data/processed_data/diurnal/all_year/{season,var,radius}.csv')   

In [6]:
def calc_diurnal(hid, radius=300, season='year'):

    temp =  pd.read_csv('/Users/lisawink/Documents/paper1/data/gap_filled_data_ta_rh.csv')
    temp['datetime_UTC']=pd.to_datetime(temp['datetime_UTC'])
    temp = temp[temp['variable']=='Ta_deg_C']
    temp = temp[temp['datetime_UTC'].dt.date.isin(hid['datetime_UTC'].dt.date)]
    temp['datetime_UTC'] = temp['datetime_UTC'].astype(str)
    temp = temp.pivot(index='station_id', columns='datetime_UTC', values='value')

    params = gpd.read_parquet(f'/Users/lisawink/Documents/paper1/data/processed_data/processed_station_params_{radius}.parquet')
    params = params.set_index('station_id')
    to_remove = ['station_no','station_name','station_long_name','station_type','station_lat','station_lon','station_elevation','mounting_structure','sky_view_factor','dominant_land_use','local_climate_zone','urban_atlas_class','urban_atlas_code','geometry','SVF_3D']
    params = params.drop(to_remove, axis=1)
    vars = params.columns

    vars = ['BuAre_median','BuVol_3D_median','BuEWA_3D_median','BuSWR_median','BuHt_wmean','BuHt_max','StrHW_mean','SVF_3D_mean','BuERI_mode']

    for var in vars:
        params_v = params[[var]]

        stats_dict = {}

        for time in temp.columns:
            params_t = params_v.merge(temp[time], left_on='station_id', right_on='station_id',how='inner')
            stats = main.calculate_statistics(params_t, time)
            stats.index = stats['Parameter']
            stats['Time'] = time
            stats_dict[time] = stats
        pd.concat(stats_dict).to_csv(f'/Users/lisawink/Documents/paper1/data/processed_data/diurnal/all_year/{season,var,radius}.csv')   

In [7]:
temp = pd.read_csv('/Users/lisawink/Documents/paper1/data/gap_filled_data_ta_rh.csv')
temp['datetime_UTC']=pd.to_datetime(temp['datetime_UTC'])
calc_diurnal(temp[['datetime_UTC']], radius=200, season='year')

KeyboardInterrupt: 

### BuAre_sum

In [41]:
var = 'BuAre_sum'
year = 'year'
radius = 300

In [42]:
stats_dict = pd.read_csv(f'/Users/lisawink/Documents/paper1/data/processed_data/altitude_out/2024/{year,var,radius}.csv') 

In [44]:
stats_dict = stats_dict.drop(columns=['Unnamed: 0','Parameter.1'])

In [50]:

lon = 7.85222
lat = 47.9959

times = {}

stats_dict['Time'] = pd.to_datetime(stats_dict['Time'])

for i in stats_dict['Time'].unique():

    times[i] = get_times(i, lon, lat)

stats_dict['sunrise'] = stats_dict['Time'].apply(
    lambda x: times[x]['sunrise'].replace(tzinfo=timezone.utc))
stats_dict['sunset'] = stats_dict['Time'].apply(
    lambda x: times[x]['sunset'].replace(tzinfo=timezone.utc))

stats_dict['time_of_day'] = np.where(
    stats_dict['Time'] < stats_dict['sunrise'], 'night',
    np.where(stats_dict['Time'] > stats_dict['sunset'], 'night', 'day')

In [53]:
stats_dict['time_of_day'] = np.where(
    stats_dict['Time'] < stats_dict['sunrise'], 'night',
    np.where(stats_dict['Time'] > stats_dict['sunset'], 'night', 'day')
)

In [67]:
vars_to_plot = ['BuAre_sum', 'BuVol_3D_sum', 'BuEWA_3D_sum', 'BuIBD', 'BuAdj', 'BuSWR_3D_median', 'BuHt_wmean','StrHW_median','SVF_3D_mean', 'BuERI_mode', 'StrClo400_median']

In [22]:
import numpy as np
from scipy.stats import spearmanr
from statsmodels.stats.multitest import multipletests

# Example: suppose you have data for many time steps
# X[t], Y[t] contain the values at time t
# You'll compute correlations per hour

p_values = stats_dict_n['Spearman p-value'].values

# Run FDR correction (Benjamini-Hochberg, common default)
reject, pvals_corrected, _, _ = multipletests(p_values, alpha=0.05, method='fdr_bh')


In [57]:
pvals_corrected

array([1.02275382e-05, 3.35516914e-06, 2.35532962e-06, ...,
       1.81767784e-05, 9.89491938e-06, 3.35678158e-05])

In [36]:
pvals_corrected.mean()

0.02940441296575158

In [29]:
p_values < pvals_corrected

array([ True,  True,  True, ...,  True,  True,  True])

In [59]:
sum(p_values < 0.05)

1987

In [31]:
len(p_values)

2193

In [30]:
# count how many are true p_values < pvals_corrected
sum(p_values < pvals_corrected)


2192

In [58]:
# check how many are significant after correction
len(pvals_corrected[pvals_corrected < 0.05])


1975

In [44]:
for i in vars_to_plot:
    stats_dict = pd.read_csv(f'/Users/lisawink/Documents/paper1/data/processed_data/diurnal/all_year/{year,i,radius}.csv') 
    print('Spearman corr > 0.6',i,len(stats_dict[(stats_dict['Spearman Correlation'] > 0.6) | (stats_dict['Spearman Correlation'] < -0.6)]) / len(stats_dict))
    print('Spearman corr > 0.7',i,len(stats_dict[(stats_dict['Spearman Correlation'] > 0.7) | (stats_dict['Spearman Correlation'] < -0.7)]) / len(stats_dict))
    print('Spearman p-val < 0.05',i,len(stats_dict[(stats_dict['Spearman p-value'] < 0.005)]) / len(stats_dict))

Spearman corr > 0.6 BuAre_sum 0.3589597353712787
Spearman corr > 0.7 BuAre_sum 0.18877609216379607
Spearman p-val < 0.05 BuAre_sum 0.5585719174175887
Spearman corr > 0.6 BuVol_3D_sum 0.3575909661229611
Spearman corr > 0.7 BuVol_3D_sum 0.2030341051671039
Spearman p-val < 0.05 BuVol_3D_sum 0.5366716094445079
Spearman corr > 0.6 BuEWA_3D_sum 0.3183529143378579
Spearman corr > 0.7 BuEWA_3D_sum 0.17839625869738793
Spearman p-val < 0.05 BuEWA_3D_sum 0.5008554807801985
Spearman corr > 0.6 BuIBD 0.14143948899281397
Spearman corr > 0.7 BuIBD 0.03216607733546253
Spearman p-val < 0.05 BuIBD 0.33443595300558915
Spearman corr > 0.6 BuAdj 0.27751796509638416
Spearman corr > 0.7 BuAdj 0.14337857876126384
Spearman p-val < 0.05 BuAdj 0.4685753393407095
Spearman corr > 0.6 BuSWR_3D_median 0.17246492528801186
Spearman corr > 0.7 BuSWR_3D_median 0.05383825710049048
Spearman p-val < 0.05 BuSWR_3D_median 0.3842819664651534
Spearman corr > 0.6 BuHt_wmean 0.27443823428766967
Spearman corr > 0.7 BuHt_wmean 0.1

In [16]:
stats_dict['Time']=pd.to_datetime(stats_dict['Time'])

In [48]:
# nighttime hours

In [18]:
stats_dict_n = stats_dict[stats_dict['Time'].dt.hour.isin([0,1,2,21,22,23])]

In [19]:
stats_dict_d = stats_dict[stats_dict['Time'].dt.hour.isin([10,11,12,13,14,15])]

In [21]:
stats_dict_n['Spearman p-value']

0       2.324524e-06
1       5.025658e-07
2       2.899859e-07
21      7.959106e-03
22      7.492167e-03
            ...     
8758    3.541716e-02
8759    7.904454e-03
8760    4.889967e-06
8761    2.219927e-06
8762    1.063823e-05
Name: Spearman p-value, Length: 2193, dtype: float64

In [43]:
#Night

In [57]:
stats_dict[(((stats_dict['Spearman Correlation'] > 0.6) | (stats_dict['Spearman Correlation'] < -0.6))) & (stats_dict['Spearman p-value'] < 0.005)]

,Unnamed: 0,Parameter,Parameter.1,Pearson Correlation,Pearson p-value,Spearman Correlation,Spearman p-value,Mutual Information,Time
0,2022-09-01 00:00:00+00:00,BuAre_sum,BuAre_sum,0.747157,7.112184e-08,0.755485,4.197272e-08,0.331665,2022-09-01 00:00:00+00:00
1,2022-09-01 01:00:00+00:00,BuAre_sum,BuAre_sum,0.808415,8.339340e-10,0.796454,2.231764e-09,0.286936,2022-09-01 01:00:00+00:00
2,2022-09-01 02:00:00+00:00,BuAre_sum,BuAre_sum,0.802746,1.340863e-09,0.812934,5.646784e-10,0.355844,2022-09-01 02:00:00+00:00
21,2022-09-01 21:00:00+00:00,BuAre_sum,BuAre_sum,0.593337,8.633700e-05,0.602429,6.282164e-05,0.243900,2022-09-01 21:00:00+00:00
24,2022-09-02 00:00:00+00:00,BuAre_sum,BuAre_sum,0.562458,2.374463e-04,0.605056,5.720527e-05,0.213615,2022-09-02 00:00:00+00:00
...,...,...,...,...,...,...,...,...,...
8737,2023-08-31 01:00:00+00:00,BuAre_sum,BuAre_sum,0.619118,3.415143e-05,0.637120,1.699488e-05,0.125482,2023-08-31 01:00:00+00:00
8757,2023-08-31 21:00:00+00:00,BuAre_sum,BuAre_sum,0.544005,4.150709e-04,0.674661,3.401271e-06,0.161121,2023-08-31 21:00:00+00:00
8760,2023-09-01 00:00:00+00:00,BuAre_sum,BuAre_sum,0.635418,1.818814e-05,0.747483,6.969922e-08,0.428086,2023-09-01 00:00:00+00:00
8761,2023-09-01 01:00:00+00:00,BuAre_sum,BuAre_sum,0.674023,3.502211e-06,0.746290,7.504980e-08,0.405092,2023-09-01 01:00:00+00:00


In [ ]:
vars = ['BuAre_sum', 'BuVol_3D_sum', 'BuEWA_3D_sum', 'BuIBD', 'BuAdj', 'BuSWR_3D_median', 'BuHt_wmean','StrHW_median','SVF_3D_mean', 'BuERI_mode', 'StrClo400_median','BuAre_median','BuVol_3D_median','BuEWA_3D_median','BuSWR_median','BuHt_max','StrHW_mean','BuERI_wmean']

In [68]:
lon = 7.85222
lat = 47.9959

for j in vars_to_plot:
    stats_dict = pd.read_csv(f'/Users/lisawink/Documents/paper1/data/processed_data/altitude_out/2024/{year,j,radius}.csv') 
    stats_dict['Time']=pd.to_datetime(stats_dict['Time'])

    times = {}

    stats_dict['Time'] = pd.to_datetime(stats_dict['Time'])

    for i in stats_dict['Time'].unique():

        times[i] = get_times(i, lon, lat)

    stats_dict['sunrise'] = stats_dict['Time'].apply(
        lambda x: times[x]['sunrise'].replace(tzinfo=timezone.utc))
    stats_dict['sunset'] = stats_dict['Time'].apply(
        lambda x: times[x]['sunset'].replace(tzinfo=timezone.utc))

    stats_dict['time_of_day'] = np.where(
        stats_dict['Time'] < stats_dict['sunrise'], 'night',
        np.where(stats_dict['Time'] > stats_dict['sunset'], 'night', 'day'))
    
    stats_dict = stats_dict[stats_dict['time_of_day'].isin(['night'])]

    p_values = stats_dict['Spearman p-value'].values

    # Run FDR correction (Benjamini-Hochberg, common default)
    reject, pvals_corrected, _, _ = multipletests(p_values, alpha=0.05, method='fdr_bh')

    stats_dict['Spearman p-value corrected'] = pvals_corrected

    print('Spearman corr > 0.5',j,len(stats_dict[(((stats_dict['Spearman Correlation'] > 0.5) | (stats_dict['Spearman Correlation'] < -0.5))) & (stats_dict['Spearman p-value corrected'] < 0.05)]) / len(stats_dict))
    #print('Spearman corr > 0.7',i,len(stats_dict[(stats_dict['Spearman Correlation'] > 0.7) | (stats_dict['Spearman Correlation'] < -0.7)]) / len(stats_dict))
    #print('Spearman p-val < 0.05',i,len(stats_dict[(stats_dict['Spearman p-value'] < 0.005)]) / len(stats_dict))

Spearman corr > 0.5 BuAre_sum 0.8066465256797583
Spearman corr > 0.5 BuVol_3D_sum 0.8064141296769696
Spearman corr > 0.5 BuEWA_3D_sum 0.802231001626772
Spearman corr > 0.5 BuIBD 0.6296769695561236
Spearman corr > 0.5 BuAdj 0.7670230072042761
Spearman corr > 0.5 BuSWR_3D_median 0.6056239832674878
Spearman corr > 0.5 BuHt_wmean 0.7275156867301883
Spearman corr > 0.5 StrHW_median 0.7974668835696026
Spearman corr > 0.5 SVF_3D_mean 0.7269346967232163
Spearman corr > 0.5 BuERI_mode 0.35242853822914244
Spearman corr > 0.5 StrClo400_median 0.07227515686730188


In [70]:
vars = ['BuAre_median','BuVol_3D_median','BuEWA_3D_median','BuSWR_median','BuHt_max','StrHW_mean','BuERI_wmean']

lon = 7.85222
lat = 47.9959

for j in vars:
    stats_dict = pd.read_csv(f'/Users/lisawink/Documents/paper1/data/processed_data/altitude_out/2024/{year,j,radius}.csv') 
    stats_dict['Time']=pd.to_datetime(stats_dict['Time'])

    times = {}

    stats_dict['Time'] = pd.to_datetime(stats_dict['Time'])

    for i in stats_dict['Time'].unique():

        times[i] = get_times(i, lon, lat)

    stats_dict['sunrise'] = stats_dict['Time'].apply(
        lambda x: times[x]['sunrise'].replace(tzinfo=timezone.utc))
    stats_dict['sunset'] = stats_dict['Time'].apply(
        lambda x: times[x]['sunset'].replace(tzinfo=timezone.utc))

    stats_dict['time_of_day'] = np.where(
        stats_dict['Time'] < stats_dict['sunrise'], 'night',
        np.where(stats_dict['Time'] > stats_dict['sunset'], 'night', 'day'))
    
    stats_dict = stats_dict[stats_dict['time_of_day'].isin(['night'])]

    p_values = stats_dict['Spearman p-value'].values

    # Run FDR correction (Benjamini-Hochberg, common default)
    reject, pvals_corrected, _, _ = multipletests(p_values, alpha=0.05, method='fdr_bh')

    stats_dict['Spearman p-value corrected'] = pvals_corrected

    print('Spearman corr > 0.5',j,len(stats_dict[(((stats_dict['Spearman Correlation'] > 0.5) | (stats_dict['Spearman Correlation'] < -0.5))) & (stats_dict['Spearman p-value corrected'] < 0.05)]) / len(stats_dict))
    #print('Spearman corr > 0.7',i,len(stats_dict[(stats_dict['Spearman Correlation'] > 0.7) | (stats_dict['Spearman Correlation'] < -0.7)]) / len(stats_dict))
    #print('Spearman p-val < 0.05',i,len(stats_dict[(stats_dict['Spearman p-value'] < 0.005)]) / len(stats_dict))

Spearman corr > 0.5 BuAre_median 0.04961654659539856
Spearman corr > 0.5 BuVol_3D_median 0.12468045549616547
Spearman corr > 0.5 BuEWA_3D_median 0.0
Spearman corr > 0.5 BuSWR_median 0.6693004880316059
Spearman corr > 0.5 BuHt_max 0.4589821055077853
Spearman corr > 0.5 StrHW_mean 0.7777132233325587
Spearman corr > 0.5 BuERI_wmean 0.060887752730653034


In [84]:
for i in vars:
    stats_dict = pd.read_csv(f'/Users/lisawink/Documents/paper1/data/processed_data/diurnal/all_year/{year,i,radius}.csv') 
    stats_dict['Time']=pd.to_datetime(stats_dict['Time'])
    stats_dict = stats_dict[stats_dict['Time'].dt.hour.isin([0,1,2,21,22,23])]
    print('Spearman corr > 0.6',i,len(stats_dict[(((stats_dict['Spearman Correlation'] > 0.5) | (stats_dict['Spearman Correlation'] < -0.5))) & (stats_dict['Spearman p-value'] < 0.005)]) / len(stats_dict))
    #print('Spearman corr > 0.7',i,len(stats_dict[(stats_dict['Spearman Correlation'] > 0.7) | (stats_dict['Spearman Correlation'] < -0.7)]) / len(stats_dict))
    #print('Spearman p-val < 0.05',i,len(stats_dict[(stats_dict['Spearman p-value'] < 0.005)]) / len(stats_dict))

Spearman corr > 0.6 BuAre_median 0.19288645690834474
Spearman corr > 0.6 BuVol_3D_median 0.2270861833105335
Spearman corr > 0.6 BuEWA_3D_median 0.005015959872321021
Spearman corr > 0.6 BuSWR_median 0.6165070679434564
Spearman corr > 0.6 BuHt_wmean 0.6853625170998632
Spearman corr > 0.6 BuHt_max 0.5303237574099408
Spearman corr > 0.6 StrHW_mean 0.674874601003192
Spearman corr > 0.6 SVF_3D_mean 0.66484268125855
Spearman corr > 0.6 BuERI_mode 0.3875968992248062


In [46]:
# Day

In [13]:
radius

200

In [16]:
# Day (200m)
radius = 200

for i in vars_to_plot:
    stats_dict = pd.read_csv(f'/Users/lisawink/Documents/paper1/data/processed_data/diurnal/all_year/{year,i,radius}.csv') 
    stats_dict['Time']=pd.to_datetime(stats_dict['Time'])
    stats_dict = stats_dict[stats_dict['Time'].dt.hour.isin([7,8,9,10,11])]
    print('Spearman corr > 0.3',i,len(stats_dict[(((stats_dict['Spearman Correlation'] > 0.3) | (stats_dict['Spearman Correlation'] < -0.3))) & (stats_dict['Spearman p-value'] < 0.005)]) / len(stats_dict))
    #print('Spearman corr > 0.7',i,len(stats_dict[(stats_dict['Spearman Correlation'] > 0.7) | (stats_dict['Spearman Correlation'] < -0.7)]) / len(stats_dict))
    #print('Spearman p-val < 0.05',i,len(stats_dict[(stats_dict['Spearman p-value'] < 0.005)]) / len(stats_dict))

Spearman corr > 0.3 BuAre_sum 0.06575342465753424
Spearman corr > 0.3 BuVol_3D_sum 0.06684931506849315
Spearman corr > 0.3 BuEWA_3D_sum 0.06301369863013699
Spearman corr > 0.3 BuIBD 0.03232876712328767
Spearman corr > 0.3 BuAdj 0.060821917808219175
Spearman corr > 0.3 BuSWR_3D_median 0.06958904109589041
Spearman corr > 0.3 BuHt_wmean 0.044931506849315066
Spearman corr > 0.3 StrHW_median 0.08
Spearman corr > 0.3 SVF_3D_mean 0.04986301369863014


FileNotFoundError: [Errno 2] No such file or directory: "/Users/lisawink/Documents/paper1/data/processed_data/diurnal/all_year/('year', 'BuERI_wmean', 200).csv"

In [17]:
stats_dict

,Unnamed: 0,Parameter,Parameter.1,Pearson Correlation,Pearson p-value,Spearman Correlation,Spearman p-value,Mutual Information,Time
7,2022-09-01 07:00:00+00:00,SVF_3D_mean,SVF_3D_mean,-0.092747,0.619723,-0.120778,0.517498,0.000000,2022-09-01 07:00:00+00:00
8,2022-09-01 08:00:00+00:00,SVF_3D_mean,SVF_3D_mean,0.051538,0.783053,0.063918,0.732649,0.000000,2022-09-01 08:00:00+00:00
9,2022-09-01 09:00:00+00:00,SVF_3D_mean,SVF_3D_mean,0.254308,0.167413,0.242766,0.188205,0.000000,2022-09-01 09:00:00+00:00
10,2022-09-01 10:00:00+00:00,SVF_3D_mean,SVF_3D_mean,0.218329,0.238021,0.245161,0.183749,0.000000,2022-09-01 10:00:00+00:00
11,2022-09-01 11:00:00+00:00,SVF_3D_mean,SVF_3D_mean,0.219517,0.235413,0.179032,0.335215,0.016635,2022-09-01 11:00:00+00:00
...,...,...,...,...,...,...,...,...,...
8743,2023-08-31 07:00:00+00:00,SVF_3D_mean,SVF_3D_mean,0.035686,0.848848,0.066129,0.723753,0.248793,2023-08-31 07:00:00+00:00
8744,2023-08-31 08:00:00+00:00,SVF_3D_mean,SVF_3D_mean,0.066535,0.722123,0.073790,0.693210,0.254496,2023-08-31 08:00:00+00:00
8745,2023-08-31 09:00:00+00:00,SVF_3D_mean,SVF_3D_mean,0.091740,0.623545,0.098810,0.596916,0.007693,2023-08-31 09:00:00+00:00
8746,2023-08-31 10:00:00+00:00,SVF_3D_mean,SVF_3D_mean,0.099669,0.593716,0.192377,0.299819,0.000000,2023-08-31 10:00:00+00:00


In [18]:
#Day (300m)
radius = 300
for i in vars_to_plot:
    stats_dict = pd.read_csv(f'/Users/lisawink/Documents/paper1/data/processed_data/diurnal/all_year/{year,i,radius}.csv') 
    stats_dict['Time']=pd.to_datetime(stats_dict['Time'])
    stats_dict = stats_dict[stats_dict['Time'].dt.hour.isin([7,8,9,10,11])]
    print('Spearman corr > 0.3',i,len(stats_dict[(((stats_dict['Spearman Correlation'] > 0.3) | (stats_dict['Spearman Correlation'] < -0.3))) & (stats_dict['Spearman p-value'] < 0.005)]) / len(stats_dict))
    #print('Spearman corr > 0.7',i,len(stats_dict[(stats_dict['Spearman Correlation'] > 0.7) | (stats_dict['Spearman Correlation'] < -0.7)]) / len(stats_dict))
    #print('Spearman p-val < 0.05',i,len(stats_dict[(stats_dict['Spearman p-value'] < 0.005)]) / len(stats_dict))

Spearman corr > 0.3 BuAre_sum 0.19726027397260273
Spearman corr > 0.3 BuVol_3D_sum 0.1736986301369863
Spearman corr > 0.3 BuEWA_3D_sum 0.16602739726027396
Spearman corr > 0.3 BuIBD 0.10849315068493151
Spearman corr > 0.3 BuAdj 0.16821917808219178
Spearman corr > 0.3 BuSWR_3D_median 0.136986301369863
Spearman corr > 0.3 BuHt_wmean 0.15178082191780823
Spearman corr > 0.3 StrHW_median 0.15013698630136987
Spearman corr > 0.3 SVF_3D_mean 0.13534246575342465
Spearman corr > 0.3 BuERI_wmean 0.09698630136986301
Spearman corr > 0.3 StrClo400_median 0.028493150684931506


In [19]:
stats_dict

,Unnamed: 0,Parameter,Parameter.1,Pearson Correlation,Pearson p-value,Spearman Correlation,Spearman p-value,Mutual Information,Time
7,2022-09-01 07:00:00+00:00,StrClo400_median,StrClo400_median,0.375532,0.024008,0.351760,0.035393,0.000000,2022-09-01 07:00:00+00:00
8,2022-09-01 08:00:00+00:00,StrClo400_median,StrClo400_median,0.361013,0.030528,0.243001,0.153274,0.000000,2022-09-01 08:00:00+00:00
9,2022-09-01 09:00:00+00:00,StrClo400_median,StrClo400_median,0.138318,0.421111,0.163975,0.339267,0.000000,2022-09-01 09:00:00+00:00
10,2022-09-01 10:00:00+00:00,StrClo400_median,StrClo400_median,0.072427,0.674648,0.126898,0.460813,0.010805,2022-09-01 10:00:00+00:00
11,2022-09-01 11:00:00+00:00,StrClo400_median,StrClo400_median,0.082064,0.634210,0.151094,0.379057,0.000000,2022-09-01 11:00:00+00:00
...,...,...,...,...,...,...,...,...,...
8743,2023-08-31 07:00:00+00:00,StrClo400_median,StrClo400_median,0.155460,0.365273,0.024710,0.886249,0.000000,2023-08-31 07:00:00+00:00
8744,2023-08-31 08:00:00+00:00,StrClo400_median,StrClo400_median,0.250314,0.140903,0.157786,0.358053,0.090911,2023-08-31 08:00:00+00:00
8745,2023-08-31 09:00:00+00:00,StrClo400_median,StrClo400_median,0.131755,0.443687,0.099118,0.565196,0.000000,2023-08-31 09:00:00+00:00
8746,2023-08-31 10:00:00+00:00,StrClo400_median,StrClo400_median,0.010431,0.951856,0.031407,0.855712,0.121908,2023-08-31 10:00:00+00:00


In [85]:
for i in vars:
    stats_dict = pd.read_csv(f'/Users/lisawink/Documents/paper1/data/processed_data/diurnal/all_year/{year,i,radius}.csv') 
    stats_dict['Time']=pd.to_datetime(stats_dict['Time'])
    stats_dict = stats_dict[stats_dict['Time'].dt.hour.isin([7,8,9,10,11])]
    print('Spearman corr > 0.3',i,len(stats_dict[(((stats_dict['Spearman Correlation'] > 0.3) | (stats_dict['Spearman Correlation'] < -0.3))) & (stats_dict['Spearman p-value'] < 0.005)]) / len(stats_dict))
    #print('Spearman corr > 0.7',i,len(stats_dict[(stats_dict['Spearman Correlation'] > 0.7) | (stats_dict['Spearman Correlation'] < -0.7)]) / len(stats_dict))
    #print('Spearman p-val < 0.05',i,len(stats_dict[(stats_dict['Spearman p-value'] < 0.005)]) / len(stats_dict))

Spearman corr > 0.3 BuAre_median 0.040547945205479455
Spearman corr > 0.3 BuVol_3D_median 0.03726027397260274
Spearman corr > 0.3 BuEWA_3D_median 0.0032876712328767125
Spearman corr > 0.3 BuSWR_median 0.16219178082191782
Spearman corr > 0.3 BuHt_wmean 0.15178082191780823
Spearman corr > 0.3 BuHt_max 0.13534246575342465
Spearman corr > 0.3 StrHW_mean 0.14904109589041095
Spearman corr > 0.3 SVF_3D_mean 0.13534246575342465
Spearman corr > 0.3 BuERI_mode 0.059726027397260274


In [78]:
for i in vars_to_plot:
    stats_dict = pd.read_csv(f'/Users/lisawink/Documents/paper1/data/processed_data/diurnal/all_year/{year,i,radius}.csv') 
    stats_dict['Time']=pd.to_datetime(stats_dict['Time'])
    stats_dict = stats_dict[stats_dict['Time'].dt.hour.isin([7,8,9,10,11])]
    print('Spearman corr > 0.6',i,len(stats_dict[(stats_dict['Spearman Correlation'] > 0.6) | (stats_dict['Spearman Correlation'] < -0.6)]) / len(stats_dict))
    print('Spearman corr > 0.7',i,len(stats_dict[(stats_dict['Spearman Correlation'] > 0.7) | (stats_dict['Spearman Correlation'] < -0.7)]) / len(stats_dict))
    print('Spearman p-val < 0.05',i,len(stats_dict[(stats_dict['Spearman p-value'] < 0.005)]) / len(stats_dict))

Spearman corr > 0.6 BuAre_sum 0.08602739726027397
Spearman corr > 0.7 BuAre_sum 0.030136986301369864
Spearman p-val < 0.05 BuAre_sum 0.19726027397260273
Spearman corr > 0.6 BuVol_3D_sum 0.08547945205479453
Spearman corr > 0.7 BuVol_3D_sum 0.027945205479452055
Spearman p-val < 0.05 BuVol_3D_sum 0.1736986301369863
Spearman corr > 0.6 BuEWA_3D_sum 0.08
Spearman corr > 0.7 BuEWA_3D_sum 0.030136986301369864
Spearman p-val < 0.05 BuEWA_3D_sum 0.16602739726027396
Spearman corr > 0.6 BuIBD 0.03123287671232877
Spearman corr > 0.7 BuIBD 0.007671232876712329
Spearman p-val < 0.05 BuIBD 0.10849315068493151
Spearman corr > 0.6 BuAdj 0.07013698630136986
Spearman corr > 0.7 BuAdj 0.02684931506849315
Spearman p-val < 0.05 BuAdj 0.16821917808219178
Spearman corr > 0.6 BuSWR_3D_median 0.04328767123287671
Spearman corr > 0.7 BuSWR_3D_median 0.01315068493150685
Spearman p-val < 0.05 BuSWR_3D_median 0.136986301369863
Spearman corr > 0.6 BuHt_wmean 0.0536986301369863
Spearman corr > 0.7 BuHt_wmean 0.0208219